#### Trial1. 변수삭제 x  >>  결측치 처리 x  >>  One-Hot Encoding x  >>  RandomOverSampling  >> SS  >>  PCA(n=2)

In [1]:
# 데이터 가져오기

In [2]:
socar_df = pd.read_csv("insurance_fraud_detect_data.csv")
pd.set_option('display.max_columns', len(socar_df.columns))
socar_df.head()

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,socarsave,start_hour,duration,accident_hour,repair_cost,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,0,4,2,4,1.302547e+05,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,0,1,4,4,0.000000e+00,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,0,1,4,6,2.581306e+05,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,0,3,3,5,1.341009e+06,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,0,1,1,1,5.240306e+05,1209410,1,0,0,2,0,0,0,-1,0


In [3]:
# train set, test set 나누기
train_set = socar_df[socar_df["test_set"] == 0]
test_set = socar_df[socar_df["test_set"] == 1]

In [4]:
# train set과 test set을 구분하던 'test_set'컬럼 삭제
train_set.drop("test_set", axis=1, inplace=True)
test_set.drop("test_set", axis=1, inplace=True)

In [5]:
# reset index
train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

In [6]:
# features와 target 나누기
X_features = train_set.drop("fraud_YN", axis=1)
y_target = train_set["fraud_YN"]
X_test = test_set.drop("fraud_YN", axis=1)
y_test = test_set["fraud_YN"]

In [7]:
# train-validation 분리 

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.73794
1     0.26206
Name: fraud_YN, dtype: float64
0    99.728261
1     0.271739
Name: fraud_YN, dtype: float64


In [8]:
# 샘플링 RamdomOverSampler

from imblearn.over_sampling import RandomOverSampler 

ros = RandomOverSampler(random_state=13)
X_train_over, y_train_over = ros.fit_sample(X_train, y_train)

In [9]:
X_train.shape, y_train.shape

((10303, 23), (10303,))

In [10]:
X_train_over.shape, y_train_over.shape

((20552, 23), (20552,))

In [11]:
from sklearn.pipeline import Pipeline

In [12]:
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

# 파이프라인 모델 설정

lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA(n_components=2)),
    ('clf', LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2)),
    ("clf", SVC(random_state=13, kernel='linear'))])

In [13]:
# 파이프라인 세팅설정

dt_pipe.set_params(clf__max_depth=4)
rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('clf',
                 LGBMClassifier(boost_from_average=False, n_estimators=1000,
                                num_leaves=64, random_state=13))])

In [16]:
# 성능 평가 함수

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score   
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test):
    
    accuracy = accuracy_score(y_valid,y_pred_valid) 
    precision = precision_score(y_valid,y_pred_valid) 
    recall = recall_score(y_valid,y_pred_valid) 
    f1 = f1_score(y_valid,y_pred_valid)
    roc_auc = roc_auc_score(y_valid,y_pred_valid)
    
    accuracy_t = accuracy_score(y_test,y_pred_test) 
    precision_t = precision_score(y_test,y_pred_test) 
    recall_t = recall_score(y_test,y_pred_test) 
    f1_t = f1_score(y_test,y_pred_test)
    roc_auc_t = roc_auc_score(y_test,y_pred_test)
    
    return accuracy, precision, recall, f1, roc_auc, accuracy_t, precision_t, recall_t, f1_t, roc_auc_t

In [17]:
# 결과
pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]

col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for pipe in pipes:
    clf = pipe
    clf.fit(X_train_over, y_train_over)
    
    y_pred_valid = clf.predict(X_valid)
    y_pred_test = clf.predict(X_test)
    
    tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.720109,0.006906,0.714286,0.013680,0.717205,0.518424,0.001997,0.428571,0.003976,0.473598
DecisionTree,0.736025,0.002954,0.285714,0.005848,0.511483,0.755847,0.001321,0.142857,0.002618,0.450041
RandomFore,0.996894,0.000000,0.000000,0.000000,0.499805,0.997437,0.000000,0.000000,0.000000,0.499839
LGBM,0.996894,0.000000,0.000000,0.000000,0.499805,0.995835,0.000000,0.000000,0.000000,0.499037
SVC,0.758929,0.006431,0.571429,0.012719,0.665434,0.527715,0.002037,0.428571,0.004054,0.478255


#### Trial2. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> RandomOverSampling >> SS >> PCA(n=3)

In [18]:
# 파이프라인 설정

lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA(n_components=3)),
    ('clf', LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3)),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3)),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3)),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=3)),
    ("clf", SVC(random_state=13, kernel='linear'))])

In [19]:
# 파이프라인 세팅설정

dt_pipe.set_params(clf__max_depth=4)
rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=3)),
                ('clf',
                 LGBMClassifier(boost_from_average=False, n_estimators=1000,
                                num_leaves=64, random_state=13))])

In [20]:
# 결과
pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]

col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for pipe in pipes:
    clf = pipe
    clf.fit(X_train_over, y_train_over)
    
    y_pred_valid = clf.predict(X_valid)
    y_pred_test = clf.predict(X_test)
    
    tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.709239,0.006649,0.714286,0.013175,0.711756,0.523230,0.002017,0.428571,0.004016,0.476007
DecisionTree,0.890528,0.003610,0.142857,0.007042,0.517711,0.862544,0.000000,0.000000,0.000000,0.432241
RandomFore,0.997283,0.000000,0.000000,0.000000,0.500000,0.997757,0.000000,0.000000,0.000000,0.500000
LGBM,0.996506,0.000000,0.000000,0.000000,0.499611,0.996155,0.000000,0.000000,0.000000,0.499197
SVC,0.759317,0.006441,0.571429,0.012739,0.665629,0.528036,0.002038,0.428571,0.004057,0.478415


#### Trial3. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> RandomOverSampling >> SS >> PCA(n=4)

In [21]:
# 파이프라인 설정

lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA(n_components=4)),
    ('clf', LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=4)),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=4)),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=4)),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=4)),
    ("clf", SVC(random_state=13, kernel='linear'))])

In [22]:
# 파이프라인 세팅설정

dt_pipe.set_params(clf__max_depth=4)
rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=4)),
                ('clf',
                 LGBMClassifier(boost_from_average=False, n_estimators=1000,
                                num_leaves=64, random_state=13))])

In [23]:
# 결과
pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]

col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for pipe in pipes:
    clf = pipe
    clf.fit(X_train_over, y_train_over)
    
    y_pred_valid = clf.predict(X_valid)
    y_pred_test = clf.predict(X_test)
    
    tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.709627,0.006658,0.714286,0.013193,0.711950,0.523550,0.002688,0.571429,0.005351,0.547436
DecisionTree,0.902174,0.004049,0.142857,0.007874,0.523550,0.868311,0.002463,0.142857,0.004843,0.506400
RandomFore,0.996894,0.000000,0.000000,0.000000,0.499805,0.997757,0.000000,0.000000,0.000000,0.500000
LGBM,0.996118,0.000000,0.000000,0.000000,0.499416,0.995835,0.000000,0.000000,0.000000,0.499037
SVC,0.757376,0.006390,0.571429,0.012638,0.664656,0.527075,0.002034,0.428571,0.004049,0.477934


#### Trial4. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> RandomOverSampling >> SS >> PCA(n=5)

In [24]:
lr_pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('pca', PCA(n_components=5)),
    ('clf', LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", SVC(random_state=13, kernel='linear'))])

In [25]:
# 파이프라인 세팅설정

dt_pipe.set_params(clf__max_depth=4)
rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=5)),
                ('clf',
                 LGBMClassifier(boost_from_average=False, n_estimators=1000,
                                num_leaves=64, random_state=13))])

In [26]:
# 결과
pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]

col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for pipe in pipes:
    clf = pipe
    clf.fit(X_train_over, y_train_over)
    
    y_pred_valid = clf.predict(X_valid)
    y_pred_test = clf.predict(X_test)
    
    tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.664208,0.004619,0.571429,0.009164,0.617945,0.589555,0.001565,0.285714,0.003113,0.437976
DecisionTree,0.904115,0.004132,0.142857,0.008032,0.524523,0.884973,0.000000,0.000000,0.000000,0.443481
RandomFore,0.996894,0.000000,0.000000,0.000000,0.499805,0.997757,0.000000,0.000000,0.000000,0.500000
LGBM,0.996894,0.000000,0.000000,0.000000,0.499805,0.996796,0.000000,0.000000,0.000000,0.499518
SVC,0.778339,0.006993,0.571429,0.013817,0.675165,0.646908,0.001820,0.285714,0.003617,0.466717


#### Trial5. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> SMOTE >> SS >> PCA

In [27]:
# 샘플링 SMOTE

from imblearn.over_sampling import SMOTE 

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)

In [28]:
X_train.shape, y_train.shape

((10303, 23), (10303,))

In [29]:
X_train_over.shape, y_train_over.shape

((20552, 23), (20552,))

In [30]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for i in range(2, 6):
    
    lr_pipe = Pipeline([
        ('scaler', StandardScaler()), 
        ('pca', PCA(n_components=i)),
        ('clf', LogisticRegression(random_state=13))])

    dt_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", DecisionTreeClassifier(random_state=13))])

    rf_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", RandomForestClassifier(random_state=13))])

    lgbm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", LGBMClassifier(random_state=13))])

    svm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", SVC(random_state=13, kernel='linear'))])


    # 파이프라인 세팅설정
    
    dt_pipe.set_params(clf__max_depth=4)
    rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
    lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
    
    pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
    
    for pipe in pipes:
        clf = pipe
        clf.fit(X_train_over, y_train_over)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))
    
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.739519,0.001502,0.142857,0.002972,0.442001,0.735982,0.001221,0.142857,0.002421,0.440086
DecisionTree,0.781832,0.001795,0.142857,0.003546,0.463215,0.772188,0.000000,0.000000,0.000000,0.386962
RandomFore,0.961180,0.010526,0.142857,0.019608,0.553134,0.970843,0.000000,0.000000,0.000000,0.486513
LGBM,0.963509,0.011236,0.142857,0.020833,0.554301,0.965075,0.000000,0.000000,0.000000,0.483622
SVC,0.719720,0.002782,0.285714,0.005510,0.503309,0.716437,0.001136,0.142857,0.002255,0.430292
LogiReg,0.751165,0.004687,0.428571,0.009274,0.590308,0.746876,0.000000,0.000000,0.000000,0.374277
DecisionTree,0.807842,0.002041,0.142857,0.004024,0.476255,0.793015,0.001560,0.142857,0.003086,0.468667
RandomFore,0.977873,0.019231,0.142857,0.033898,0.561503,0.982698,0.000000,0.000000,0.000000,0.492453
LGBM,0.973991,0.016129,0.142857,0.028986,0.559556,0.976930,0.000000,0.000000,0.000000,0.489563
SVC,0.737578,0.004444,0.428571,0.008798,0.583496,0.728933,0.001189,0.142857,0.002358,0.436554


#### Trial5. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> ADASYN >> SS >> PCA

In [31]:
# 샘플링 ADASYN

from imblearn.over_sampling import ADASYN

ads = SMOTE(random_state=13)
X_train_over, y_train_over = ads.fit_sample(X_train, y_train)

In [32]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for i in range(2, 6):
    
    # 파이프라인 설정
    
    lr_pipe = Pipeline([
        ('scaler', StandardScaler()), 
        ('pca', PCA(n_components=i)),
        ('clf', LogisticRegression(random_state=13))])

    dt_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", DecisionTreeClassifier(random_state=13))])

    rf_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", RandomForestClassifier(random_state=13))])

    lgbm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", LGBMClassifier(random_state=13))])

    svm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", SVC(random_state=13, kernel='linear'))])


    # 파이프라인 세팅설정
    
    dt_pipe.set_params(clf__max_depth=4)
    rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
    lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
    
    pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
    
    for pipe in pipes:
        clf = pipe
        clf.fit(X_train_over, y_train_over)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

# 결과   
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.739907,0.001504,0.142857,0.002976,0.442195,0.735662,0.001220,0.142857,0.002418,0.439926
DecisionTree,0.781056,0.001789,0.142857,0.003534,0.462826,0.771548,0.000000,0.000000,0.000000,0.386641
RandomFore,0.962345,0.010870,0.142857,0.020202,0.553717,0.969561,0.000000,0.000000,0.000000,0.485870
LGBM,0.961180,0.010526,0.142857,0.019608,0.553134,0.967639,0.000000,0.000000,0.000000,0.484907
SVC,0.718944,0.002774,0.285714,0.005495,0.502919,0.716437,0.001136,0.142857,0.002255,0.430292
LogiReg,0.751165,0.004687,0.428571,0.009274,0.590308,0.746235,0.000000,0.000000,0.000000,0.373956
DecisionTree,0.774457,0.001736,0.142857,0.003431,0.459517,0.772188,0.001416,0.142857,0.002805,0.458230
RandomFore,0.979814,0.021277,0.142857,0.037037,0.562476,0.980775,0.000000,0.000000,0.000000,0.491490
LGBM,0.971273,0.014493,0.142857,0.026316,0.558194,0.978212,0.000000,0.000000,0.000000,0.490206
SVC,0.736025,0.004418,0.428571,0.008746,0.582717,0.729253,0.001190,0.142857,0.002361,0.436714


#### Trial6. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> RandomUnder >> SS >> PCA

In [33]:
# 샘플링 RandomUnderSampler

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=13)
X_train_under, y_train_under = rus.fit_sample(X_train, y_train)

In [34]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for i in range(2, 6):
    
    # 파이프라인 설정
    
    lr_pipe = Pipeline([
        ('scaler', StandardScaler()), 
        ('pca', PCA(n_components=i)),
        ('clf', LogisticRegression(random_state=13))])

    dt_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", DecisionTreeClassifier(random_state=13))])

    rf_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", RandomForestClassifier(random_state=13))])

    lgbm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", LGBMClassifier(random_state=13))])

    svm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", SVC(random_state=13, kernel='linear'))])


    # 파이프라인 세팅설정
    
    dt_pipe.set_params(clf__max_depth=4)
    rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
    lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
    
    pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
    
    for pipe in pipes:
        clf = pipe
        clf.fit(X_train_under, y_train_under)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

# 결과   
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.658385,0.004540,0.571429,0.009009,0.615025,0.459468,0.002370,0.571429,0.004720,0.515323
DecisionTree,0.536879,0.003350,0.571429,0.006661,0.554107,0.508170,0.001956,0.428571,0.003894,0.468460
RandomFore,0.560947,0.004409,0.714286,0.008764,0.637408,0.485101,0.001868,0.428571,0.003720,0.456900
LGBM,0.535326,0.004167,0.714286,0.008285,0.624562,0.433515,0.001698,0.428571,0.003382,0.431049
SVC,0.703804,0.005236,0.571429,0.010376,0.637797,0.516822,0.001991,0.428571,0.003963,0.472796
LogiReg,0.656444,0.004515,0.571429,0.008959,0.614052,0.457546,0.002361,0.571429,0.004703,0.514359
DecisionTree,0.729814,0.005739,0.571429,0.011364,0.650837,0.645947,0.001815,0.285714,0.003607,0.466235
RandomFore,0.613742,0.004016,0.571429,0.007976,0.592643,0.499840,0.001284,0.285714,0.002556,0.393018
LGBM,0.509705,0.004732,0.857143,0.009412,0.682951,0.396988,0.002124,0.571429,0.004233,0.484012
SVC,0.703804,0.005236,0.571429,0.010376,0.637797,0.516822,0.001991,0.428571,0.003963,0.472796


#### Trial7. 변수삭제 x >> 결측치 처리 x >> One-Hot Encoding x >> SMOTENN >> SS >> PCA

In [35]:
# 샘플링 SMOTENN

from imblearn.combine import SMOTEENN

smt = SMOTEENN(random_state=13)
X_train_comb, y_train_comb = smt.fit_sample(X_train, y_train)

In [36]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for i in range(2, 6):
    
    # 파이프라인 설정
    
    lr_pipe = Pipeline([
        ('scaler', StandardScaler()), 
        ('pca', PCA(n_components=i)),
        ('clf', LogisticRegression(random_state=13))])

    dt_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", DecisionTreeClassifier(random_state=13))])

    rf_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", RandomForestClassifier(random_state=13))])

    lgbm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", LGBMClassifier(random_state=13))])

    svm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", SVC(random_state=13, kernel='linear'))])


    # 파이프라인 세팅설정
    
    dt_pipe.set_params(clf__max_depth=4)
    rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
    lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
    
    pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
    
    for pipe in pipes:
        clf = pipe
        clf.fit(X_train_comb, y_train_comb)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

# 결과   
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.745730,0.001538,0.142857,0.003044,0.445115,0.737584,0.000000,0.000000,0.000000,0.369621
DecisionTree,0.728649,0.002874,0.285714,0.005690,0.507785,0.713874,0.000000,0.000000,0.000000,0.357739
RandomFore,0.965450,0.000000,0.000000,0.000000,0.484040,0.961871,0.000000,0.000000,0.000000,0.482017
LGBM,0.961180,0.000000,0.000000,0.000000,0.481900,0.960590,0.000000,0.000000,0.000000,0.481374
SVC,0.732531,0.002915,0.285714,0.005772,0.509731,0.720602,0.000000,0.000000,0.000000,0.361111
LogiReg,0.764363,0.004950,0.428571,0.009788,0.596925,0.755207,0.001318,0.142857,0.002611,0.449720
DecisionTree,0.776398,0.003490,0.285714,0.006897,0.531724,0.738225,0.000000,0.000000,0.000000,0.369942
RandomFore,0.977096,0.000000,0.000000,0.000000,0.489879,0.971483,0.000000,0.000000,0.000000,0.486834
LGBM,0.974379,0.000000,0.000000,0.000000,0.488517,0.975008,0.000000,0.000000,0.000000,0.488600
SVC,0.756988,0.004800,0.428571,0.009494,0.593227,0.744954,0.001264,0.142857,0.002506,0.444582


#### Trial8. 변수삭제 x >> repair_cost(중앙값) >> One-Hot Encoding x >> RandomOverSampling >> SS >> PCA

In [37]:
socar_copy1 = socar_df.copy()

In [38]:
no_repair = socar_copy1[socar_copy1['repair_cost'] == 0]
repair = socar_copy1[socar_copy1['repair_cost'] != 0]

repair['repair_cost'].median()

303747.9937

In [39]:
socar_copy1.loc[(socar_copy1.repair_cost == 0), 'repair_cost'] = repair['repair_cost'].median()

In [40]:
# train set, test set 나누기
train_set = socar_copy1[socar_copy1["test_set"] == 0]
test_set = socar_copy1[socar_copy1["test_set"] == 1]

In [41]:
# train set과 test set을 구분하던 'test_set'컬럼 삭제
train_set.drop("test_set", axis=1, inplace=True)
test_set.drop("test_set", axis=1, inplace=True)

In [42]:
# reset index
train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

In [43]:
# features와 target 나누기
X_features = train_set.drop("fraud_YN", axis=1)
y_target = train_set["fraud_YN"]
X_test = test_set.drop("fraud_YN", axis=1)
y_test = test_set["fraud_YN"]

In [44]:
# train-validation 분리 

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.73794
1     0.26206
Name: fraud_YN, dtype: float64
0    99.728261
1     0.271739
Name: fraud_YN, dtype: float64


In [45]:
# 샘플링 RamdomOverSampler

from imblearn.over_sampling import RandomOverSampler 

ros = RandomOverSampler(random_state=13)
X_train_over, y_train_over = ros.fit_sample(X_train, y_train)

In [46]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for i in range(2, 6):
    
    # 파이프라인 설정
    
    lr_pipe = Pipeline([
        ('scaler', StandardScaler()), 
        ('pca', PCA(n_components=i)),
        ('clf', LogisticRegression(random_state=13))])

    dt_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", DecisionTreeClassifier(random_state=13))])

    rf_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", RandomForestClassifier(random_state=13))])

    lgbm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", LGBMClassifier(random_state=13))])

    svm_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", SVC(random_state=13, kernel='linear'))])


    # 파이프라인 세팅설정
    
    dt_pipe.set_params(clf__max_depth=4)
    rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
    lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
    
    pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
    
    for pipe in pipes:
        clf = pipe
        clf.fit(X_train_over, y_train_over)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

# 결과   
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.706910,0.006596,0.714286,0.013072,0.710588,0.524832,0.002024,0.428571,0.004030,0.476810
DecisionTree,0.945652,0.007407,0.142857,0.014085,0.545348,0.927267,0.004505,0.142857,0.008734,0.535944
RandomFore,0.996506,0.000000,0.000000,0.000000,0.499611,0.997437,0.000000,0.000000,0.000000,0.499839
LGBM,0.996506,0.000000,0.000000,0.000000,0.499611,0.997757,0.000000,0.000000,0.000000,0.500000
SVC,0.760093,0.006462,0.571429,0.012780,0.666018,0.528997,0.002042,0.428571,0.004065,0.478897
LogiReg,0.707686,0.006614,0.714286,0.013106,0.710977,0.526434,0.002031,0.428571,0.004043,0.477613
DecisionTree,0.704969,0.000000,0.000000,0.000000,0.353445,0.788850,0.004559,0.428571,0.009023,0.609116
RandomFore,0.997283,0.000000,0.000000,0.000000,0.500000,0.997757,0.000000,0.000000,0.000000,0.500000
LGBM,0.996506,0.000000,0.000000,0.000000,0.499611,0.996796,0.000000,0.000000,0.000000,0.499518
SVC,0.758540,0.006421,0.571429,0.012698,0.665239,0.527715,0.002037,0.428571,0.004054,0.478255


#### Trial9. 변수삭제 x >> repair_cost(중앙값) >> One-Hot Encoding x >> SMOTE >> SS >> PCA

In [153]:
def get_score(X_train, y_train):

    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
                 'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
    index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
            'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
            'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
            'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
    tmp = []
    
    for i in range(2, 6):
        
        # 파이프라인 설정
        
        lr_pipe = Pipeline([
            ('scaler', StandardScaler()), 
            ('pca', PCA(n_components=i)),
            ('clf', LogisticRegression(random_state=13))])
    
        dt_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=i)),
            ("clf", DecisionTreeClassifier(random_state=13))])
    
        rf_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=i)),
            ("clf", RandomForestClassifier(random_state=13))])
    
        lgbm_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=i)),
            ("clf", LGBMClassifier(random_state=13))])
    
        svm_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=i)),
            ("clf", SVC(random_state=13, kernel='linear'))])
    
    
        # 파이프라인 세팅설정
        
        dt_pipe.set_params(clf__max_depth=4)
        rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
        lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
        
        pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
        
        for pipe in pipes:
            clf = pipe
            clf.fit(X_train, y_train)
            
            y_pred_valid = clf.predict(X_valid)
            y_pred_test = clf.predict(X_test)
            
            tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))
    
    # 결과   
    return pd.DataFrame(tmp, columns=col_names, index=index)

In [154]:
# 샘플링 SMOTE

from imblearn.over_sampling import SMOTE 

smote = SMOTE(random_state=13)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)

In [155]:
get_score(X_train_over, y_train_over)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.737578,0.001490,0.142857,0.002950,0.441028,0.735021,0.001217,0.142857,0.002413,0.439605
DecisionTree,0.741460,0.001513,0.142857,0.002994,0.442974,0.767703,0.001389,0.142857,0.002751,0.455982
RandomFore,0.970109,0.000000,0.000000,0.000000,0.486376,0.973086,0.000000,0.000000,0.000000,0.487636
LGBM,0.959239,0.000000,0.000000,0.000000,0.480926,0.962832,0.000000,0.000000,0.000000,0.482498
SVC,0.720497,0.002789,0.285714,0.005525,0.503698,0.710990,0.001115,0.142857,0.002212,0.427562
LogiReg,0.748447,0.004637,0.428571,0.009174,0.588945,0.740147,0.000000,0.000000,0.000000,0.370906
DecisionTree,0.800466,0.001965,0.142857,0.003876,0.472557,0.811919,0.000000,0.000000,0.000000,0.406872
RandomFore,0.979425,0.000000,0.000000,0.000000,0.491047,0.980455,0.000000,0.000000,0.000000,0.491329
LGBM,0.976320,0.000000,0.000000,0.000000,0.489490,0.978853,0.000000,0.000000,0.000000,0.490527
SVC,0.738742,0.004464,0.428571,0.008837,0.584079,0.725088,0.001172,0.142857,0.002326,0.434627


In [156]:
# 샘플링 ADASYN

from imblearn.over_sampling import ADASYN

ads = SMOTE(random_state=13)
X_train_over, y_train_over = ads.fit_sample(X_train, y_train)

In [157]:
get_score(X_train_over, y_train_over)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.737966,0.001493,0.142857,0.002954,0.441222,0.735021,0.001217,0.142857,0.002413,0.439605
DecisionTree,0.741848,0.001515,0.142857,0.002999,0.443169,0.767062,0.001385,0.142857,0.002743,0.455661
RandomFore,0.969332,0.000000,0.000000,0.000000,0.485987,0.971483,0.000000,0.000000,0.000000,0.486834
LGBM,0.960404,0.000000,0.000000,0.000000,0.481510,0.963473,0.000000,0.000000,0.000000,0.482820
SVC,0.720885,0.002793,0.285714,0.005533,0.503893,0.710990,0.001115,0.142857,0.002212,0.427562
LogiReg,0.749612,0.004658,0.428571,0.009217,0.589529,0.738545,0.000000,0.000000,0.000000,0.370103
DecisionTree,0.805124,0.004008,0.285714,0.007905,0.546127,0.764819,0.000000,0.000000,0.000000,0.383269
RandomFore,0.977873,0.000000,0.000000,0.000000,0.490269,0.984620,0.000000,0.000000,0.000000,0.493417
LGBM,0.977484,0.000000,0.000000,0.000000,0.490074,0.977251,0.000000,0.000000,0.000000,0.489724
SVC,0.737966,0.004451,0.428571,0.008811,0.583690,0.723806,0.001167,0.142857,0.002315,0.433985


In [158]:
# 샘플링 ADASYN

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=13)
X_train_under, y_train_under = rus.fit_sample(X_train, y_train)

In [159]:
get_score(X_train_under, y_train_under)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.655668,0.004505,0.571429,0.008939,0.613663,0.463954,0.002389,0.571429,0.004759,0.517570
DecisionTree,0.789208,0.005535,0.428571,0.010929,0.609381,0.691125,0.002081,0.285714,0.004132,0.488875
RandomFore,0.557065,0.003503,0.571429,0.006963,0.564227,0.467478,0.001206,0.285714,0.002401,0.376801
LGBM,0.590450,0.004726,0.714286,0.009390,0.652199,0.507209,0.002599,0.571429,0.005175,0.539247
SVC,0.704193,0.005242,0.571429,0.010390,0.637991,0.516822,0.001991,0.428571,0.003963,0.472796
LogiReg,0.658385,0.004540,0.571429,0.009009,0.615025,0.465556,0.002397,0.571429,0.004773,0.518373
DecisionTree,0.795807,0.005714,0.428571,0.011278,0.612690,0.687600,0.002058,0.285714,0.004086,0.487109
RandomFore,0.670031,0.004700,0.571429,0.009324,0.620864,0.545658,0.001413,0.285714,0.002813,0.415979
LGBM,0.512034,0.004754,0.857143,0.009456,0.684118,0.408843,0.002167,0.571429,0.004317,0.489953
SVC,0.704581,0.005249,0.571429,0.010403,0.638186,0.516822,0.001991,0.428571,0.003963,0.472796


In [160]:
# 샘플링 SMOTEENN

from imblearn.combine import SMOTEENN

smt = SMOTEENN(random_state=13)
X_train_comb, y_train_comb = smt.fit_sample(X_train, y_train)

In [161]:
get_score(X_train_comb, y_train_comb)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.727873,0.001437,0.142857,0.002845,0.436162,0.718680,0.000000,0.000000,0.000000,0.360148
DecisionTree,0.741071,0.003012,0.285714,0.005961,0.514013,0.731496,0.001200,0.142857,0.002381,0.437838
RandomFore,0.960016,0.000000,0.000000,0.000000,0.481316,0.963153,0.000000,0.000000,0.000000,0.482659
LGBM,0.958851,0.000000,0.000000,0.000000,0.480732,0.956104,0.007576,0.142857,0.014388,0.550395
SVC,0.711568,0.001355,0.142857,0.002685,0.427988,0.709068,0.000000,0.000000,0.000000,0.355331
LogiReg,0.739130,0.002990,0.285714,0.005917,0.513040,0.731817,0.000000,0.000000,0.000000,0.366731
DecisionTree,0.720109,0.002786,0.285714,0.005517,0.503503,0.719000,0.001147,0.142857,0.002275,0.431576
RandomFore,0.973214,0.000000,0.000000,0.000000,0.487933,0.973726,0.000000,0.000000,0.000000,0.487958
LGBM,0.970109,0.000000,0.000000,0.000000,0.486376,0.971483,0.000000,0.000000,0.000000,0.486834
SVC,0.745730,0.004587,0.428571,0.009077,0.587583,0.728613,0.001188,0.142857,0.002356,0.436393


#### Trial10. 변수삭제 o >> repair, insure_cost(중앙값) >> One-Hot Encoding o >> RandomOverSampling >> SS >> PCA

In [171]:
socar_copy2 = socar_df.copy()

In [172]:
socar_copy2.drop(["sharing_type", "has_previous_accident", "socarpass", "car_part2", "repair_cnt", "b2b",\
                 "insurance_site_aid_YN", "police_site_aid_YN"], axis=1, inplace=True)

In [173]:
# accident_ratio 분류하기
for i, j in socar_copy2.iterrows():
    if j['accident_ratio'] < 40:
        socar_copy2.loc[i, 'accident_ratio'] = 0
    else:
        socar_copy2.loc[i, 'accident_ratio'] = 1

socar_copy2['accident_ratio'].value_counts()

1    14345
0     1655
Name: accident_ratio, dtype: int64

In [177]:
row_to_drop = socar_copy2[(socar_copy2["accident_hour"] == -1) | (socar_copy2["accident_location"] == 3) |\
           (socar_copy2["accident_location"] == 5)].index

In [178]:
socar_copy2.shape

(16000, 17)

In [179]:
socar_copy2.drop(row_to_drop, axis=0, inplace=True)
socar_copy2.shape

(15112, 17)

In [180]:
# 원핫인코딩
socar_copy2 = pd.get_dummies(socar_copy2, columns=["car_model", "age_group", "cumulative_use_count",\
                                                 "pf_type", "start_hour",\
                                                 "duration", "accident_hour", "accident_location",\
                                                 "acc_type1", "total_prsn_cnt"])

In [181]:
# train set, test set 나누기
train_set = socar_copy2[socar_copy2["test_set"] == 0]
test_set = socar_copy2[socar_copy2["test_set"] == 1]

In [182]:
# train set과 test set을 구분하던 'test_set'컬럼 삭제
train_set.drop("test_set", axis=1, inplace=True)
test_set.drop("test_set", axis=1, inplace=True)

In [183]:
# reset index
train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

In [184]:
# features와 target 나누기
X_features = train_set.drop("fraud_YN", axis=1)
y_target = train_set["fraud_YN"]
X_test = test_set.drop("fraud_YN", axis=1)
y_test = test_set["fraud_YN"]

In [185]:
# train-validation 분리 

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.721822
1     0.278178
Name: fraud_YN, dtype: float64
0    99.711578
1     0.288422
Name: fraud_YN, dtype: float64


In [186]:
# 샘플링 RamdomOverSampler

from imblearn.over_sampling import RandomOverSampler 

ros = RandomOverSampler(random_state=13)
X_train_over, y_train_over = ros.fit_sample(X_train, y_train)

In [187]:
from sklearn.preprocessing import RobustScaler

In [188]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC',\
        'LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []

for i in range(2, 6):
    
    # 파이프라인 설정
    
    lr_pipe = Pipeline([
        ('scaler', RobustScaler()), 
        ('pca', PCA(n_components=i)),
        ('clf', LogisticRegression(random_state=13))])

    dt_pipe = Pipeline([
        ('scaler', RobustScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", DecisionTreeClassifier(random_state=13))])

    rf_pipe = Pipeline([
        ('scaler', RobustScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", RandomForestClassifier(random_state=13))])

    lgbm_pipe = Pipeline([
        ('scaler', RobustScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", LGBMClassifier(random_state=13))])

    svm_pipe = Pipeline([
        ('scaler', RobustScaler()),
        ('pca', PCA(n_components=i)),
        ("clf", SVC(random_state=13, kernel='linear'))])


    # 파이프라인 세팅설정
    
    dt_pipe.set_params(clf__max_depth=4)
    rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
    lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)
    
    pipes = [lr_pipe, dt_pipe, rf_pipe, lgbm_pipe, svm_pipe]
    
    for pipe in pipes:
        clf = pipe
        clf.fit(X_train_over, y_train_over)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

# 결과   
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.741656,0.000000,0.000000,0.000000,0.371901,0.733132,0.000000,0.000000,0.000000,0.367429
DecisionTree,0.756489,0.001706,0.142857,0.003373,0.450561,0.752266,0.000000,0.000000,0.000000,0.377019
RandomFore,0.995056,0.000000,0.000000,0.000000,0.498967,0.997315,0.000000,0.000000,0.000000,0.499832
LGBM,0.995880,0.000000,0.000000,0.000000,0.499380,0.996643,0.000000,0.000000,0.000000,0.499495
SVC,0.964153,0.000000,0.000000,0.000000,0.483471,0.955018,0.000000,0.000000,0.000000,0.478634
LogiReg,0.708282,0.004243,0.428571,0.008403,0.568831,0.505203,0.002037,0.428571,0.004054,0.466978
DecisionTree,0.879687,0.003484,0.142857,0.006803,0.512338,0.848943,0.002247,0.142857,0.004425,0.496731
RandomFore,0.996704,0.000000,0.000000,0.000000,0.499793,0.997650,0.000000,0.000000,0.000000,0.500000
LGBM,0.994644,0.000000,0.000000,0.000000,0.498760,0.996643,0.000000,0.000000,0.000000,0.499495
SVC,0.724351,0.004491,0.428571,0.008889,0.576889,0.520980,0.002104,0.428571,0.004187,0.474885


In [189]:
# 파이프라인 설정
    
lr_pipe = Pipeline([('scaler', RobustScaler()), ('clf', LogisticRegression(random_state=13))])

dt_pipe = Pipeline([('scaler', RobustScaler()), ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([('scaler', RobustScaler()), ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([('scaler', RobustScaler()), ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([('scaler', RobustScaler()), ("clf", SVC(random_state=13, kernel='linear'))])


# 파이프라인 세팅설정
    
dt_pipe.set_params(clf__max_depth=4)
rf_pipe.set_params(clf__n_jobs=-1, clf__n_estimators=100)
lgbm_pipe.set_params(clf__n_estimators=1000, clf__num_leaves=64, clf__n_jobs=-1, clf__boost_from_average=False)

Pipeline(steps=[('scaler', RobustScaler()),
                ('clf',
                 LGBMClassifier(boost_from_average=False, n_estimators=1000,
                                num_leaves=64, random_state=13))])

In [190]:
col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc',\
             'Accuracy_t', 'precision_t', 'recall_t', 'f1_t', 'roc_t']    
index = ['LogiReg', 'DecisionTree', 'RandomFore', 'LGBM', 'SVC']
tmp = []


    
for pipe in pipes:
    clf = pipe
    clf.fit(X_train_over, y_train_over)
        
    y_pred_valid = clf.predict(X_valid)
    y_pred_test = clf.predict(X_test)
        
    tmp.append(get_clf_eval(y_valid, y_pred_valid, y_test, y_pred_test))

# 결과   
pd.DataFrame(tmp, columns=col_names, index=index)

,Accuracy,precision,recall,f1,roc,Accuracy_t,precision_t,recall_t,f1_t,roc_t
LogiReg,0.695509,0.004065,0.428571,0.008054,0.562426,0.513931,0.002073,0.428571,0.004127,0.471352
DecisionTree,0.822415,0.002347,0.142857,0.004619,0.483619,0.807989,0.005254,0.428571,0.010381,0.618727
RandomFore,0.996704,0.000000,0.000000,0.000000,0.499793,0.997315,0.000000,0.000000,0.000000,0.499832
LGBM,0.995468,0.000000,0.000000,0.000000,0.499174,0.996979,0.000000,0.000000,0.000000,0.499664
SVC,0.721467,0.005908,0.571429,0.011696,0.646665,0.522659,0.002111,0.428571,0.004202,0.475726
